<a href="https://colab.research.google.com/github/MohsenJadidi/Automatic-Modulation-Classification-AMC/blob/master/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Importing the dataset (Copy from CNN.ipynb and merge cells)
without Reshape(2 end line)

In [2]:
import pickle
import numpy as np
fileName = 'RML2016.10a_dict.pkl'
with open("/content/drive/My Drive/Colab Notebooks/"+fileName,'rb') as f:
  data = pickle.load(f,encoding='bytes')
 
X = []
labels = [] # label each example by a pair (modulation type, snr)
total_examples = 0
analog = [b'AM-DSB', b'AM-SSB', b'WBFM']

for mod_type, snr in data.keys():
    if (mod_type not in analog):      
        current_matrix = data[(mod_type, snr)]        
        total_examples += current_matrix.shape[0]
        for i in range(current_matrix.shape[0]):
            X.append(current_matrix[i])
            labels.append((str(mod_type, 'ascii'), snr)) # mod_type is of type bytes
    
X = np.array(X)         # First row is QPSK snr=2, seconde is PAM4 snr=8 , ...
labels = np.array(labels)

y = labels[:,0]

print(f'loaded {total_examples} signal vectors into X{X.shape} and their corresponding'
      f' labels into labels{labels.shape}')  
# print(np.unique(labels[:,0]))

# Encoding categorical data
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
labelencoder_y = LabelEncoder()
y = labelencoder_y.fit_transform(y)
onehotencoder = OneHotEncoder()
y = onehotencoder.fit_transform(y.reshape(-1,1)).toarray()

snrList = [str(2*i-20) for i in range(20)]  # snrList = -20, -18, -16 , ... ,0, ... ,18
snr = snrList[19]
numberOfEachExamples = 1000
print("SNR :", snr)

output = [[labels[i*numberOfEachExamples, 0],y[i*numberOfEachExamples]] for i in range(int(X.shape[0]/numberOfEachExamples))]
output = dict(output)

X_snr = []
y_snr = []

for i in range(X.shape[0]):
    if labels[i,1] == snr:
        X_snr.append(X[i])
        y_snr.append(y[i])

X_snr = np.array(X_snr)
y_snr = np.array(y_snr)  

# Change IQ to amplitude and phase
'''
from math import sqrt, atan2, pow, pi

AmpPhs_snr = np.zeros_like(X_snr)
for i in range(X_snr.shape[0]):
    for t in range(X_snr.shape[2]):
        AmpPhs_snr[i,0,t] = sqrt(pow(X_snr[i,0,t], 2) + pow(X_snr[i,1,t],2))
        AmpPhs_snr[i,1,t] = (atan2(X_snr[i,0,t], X_snr[i,1,t])) / pi
'''

X_cmplx = X_snr[:,0,:] + 1j* X_snr[:,1,:]
    
X_amp = np.abs(X_cmplx)
#X_amp = X_amp/(np.reshape(np.sqrt(np.mean(X_amp**2,axis=1)),(-1,1))*np.ones([1,128]))
X_ang = np.arctan2(X_snr[:,1,:],X_snr[:,0,:])
#X_ang = X_ang/np.pi
    
    
X_amp = np.reshape(X_amp,(-1,1,128))
X_ang = np.reshape(X_ang,(-1,1,128))
    
AmpPhs_snr = np.concatenate((X_amp,X_ang), axis=1) 
#AmpPhs_snr = np.transpose(np.array(AmpPhs_snr),(0,2,1))        
  
###### Splitting the dataset into the Training set and Test set ######
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(AmpPhs_snr, y_snr, test_size = 0.2, random_state = 0)
# The below line better for Cross_val part
#X_train, X_test, y_train, y_test = train_test_split(X_snr, y_snr, test_size = 1, random_state = 0)


# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# StandardScaler expected <= 2.
#X_train = X_train.reshape([6400,256])
X_train = X_train.reshape([X_train.shape[0],256])
X_train = sc.fit_transform(X_train)
X_train = X_train.reshape([X_train.shape[0],2,128])
#X_test = X_test.reshape([1600,256])
X_test = X_test.reshape([X_test.shape[0],256])
X_test = sc.transform(X_test)
X_test = X_test.reshape([X_test.shape[0],2,128])

loaded 160000 signal vectors into X(160000, 2, 128) and their corresponding labels into labels(160000, 2)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


SNR : 18


## Import Module

In [0]:
import keras
from keras.models import Sequential
from keras.models import Model
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import Input
from keras.layers import add
from keras.layers import Reshape
from keras.layers import LSTM
from keras.layers import BatchNormalization, SpatialDropout1D
from keras.layers.convolutional import ZeroPadding2D


#Making LSTM0 model
(Thesis :DEEP NEURAL NETWORK ARCHITECTURES FOR MODULATION CLASSIFICATION)

In [46]:
classifier = Sequential()


dout = 0.8
classifier.add(LSTM(128, input_shape =  (2,128), activation = 'relu'))
classifier.add(Dropout(dout))
classifier.add(Reshape(target_shape = (1,128)))
classifier.add(LSTM(128, activation = 'relu'))
classifier.add(Dropout(dout))


#classifier.add(Flatten())
#classifier.add(Dense(output_dim = 128 , activation = 'relu')) # hidden layer
#classifier.add(Dropout(dout))
classifier.add(Dense(output_dim = 8, activation = 'softmax'))

adamOpt = keras.optimizers.adam(lr=0.001)

classifier.compile(optimizer = adamOpt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

print("Model Created!")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_34 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_31 (Dropout)         (None, 128)               0         
_________________________________________________________________
reshape_6 (Reshape)          (None, 1, 128)            0         
_________________________________________________________________
lstm_35 (LSTM)               (None, 128)               131584    
_________________________________________________________________
dropout_32 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_17 (Dense)             (None, 8)                 1032      
Total params: 264,200
Trainable params: 264,200
Non-trainable params: 0
_________________________________________________________________
Mode

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`
  from ipykernel import kernelapp as app


#Making LSTM1 model

In [52]:
classifier = Sequential()


dout = 0.7
classifier.add(LSTM(128, input_shape =  (2,128), activation = 'tanh', return_sequences=True, recurrent_dropout=0.2))
classifier.add(SpatialDropout1D(0.5))
#classifier.add(BatchNormalization())
#classifier.add(Dropout(dout))
# classifier.add(Reshape(target_shape = (1,128)))
classifier.add(LSTM(128, activation = 'tanh', return_sequences=True, recurrent_dropout=0.2))
classifier.add(SpatialDropout1D(0.5))
#classifier.add(BatchNormalization())
#classifier.add(Dropout(dout))


classifier.add(Flatten())
#classifier.add(BatchNormalization())
#classifier.add(Dense(output_dim = 128 , activation = 'relu')) # hidden layer
#classifier.add(Dropout(dout))

classifier.add(Dense(output_dim = 8, activation = 'softmax'))

adamOpt = keras.optimizers.adam(lr=0.001)

classifier.compile(optimizer = adamOpt, loss = 'categorical_crossentropy', metrics = ['accuracy'])

classifier.summary()

print("Model Created!")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_40 (LSTM)               (None, 2, 32)             20608     
_________________________________________________________________
dropout_37 (Dropout)         (None, 2, 32)             0         
_________________________________________________________________
flatten_14 (Flatten)         (None, 64)                0         
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 520       
Total params: 21,128
Trainable params: 21,128
Non-trainable params: 0
_________________________________________________________________
Model Created!


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(activation="softmax", units=8)`


# Fitting model

In [53]:
batch = 64
epoch = 100
history = classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch, validation_data=(X_test, y_test))
#history = classifier.fit(X_train, y_train, batch_size = batch, epochs = epoch)

Train on 6400 samples, validate on 1600 samples
Epoch 1/100
6400/6400 [==============================] - 7s 1ms/step - loss: 2.1250 - acc: 0.1406 - val_loss: 2.0417 - val_acc: 0.2106
Epoch 2/100
6400/6400 [==============================] - 1s 191us/step - loss: 2.0371 - acc: 0.1958 - val_loss: 1.9793 - val_acc: 0.2925
Epoch 3/100
6400/6400 [==============================] - 1s 189us/step - loss: 1.9703 - acc: 0.2403 - val_loss: 1.8953 - val_acc: 0.3456
Epoch 4/100
6400/6400 [==============================] - 1s 190us/step - loss: 1.8910 - acc: 0.2823 - val_loss: 1.8004 - val_acc: 0.3706
Epoch 5/100
6400/6400 [==============================] - 1s 188us/step - loss: 1.7933 - acc: 0.3233 - val_loss: 1.6977 - val_acc: 0.3975
Epoch 6/100
6400/6400 [==============================] - 1s 190us/step - loss: 1.7126 - acc: 0.3416 - val_loss: 1.5988 - val_acc: 0.4269
Epoch 7/100
6400/6400 [==============================] - 1s 188us/step - loss: 1.6272 - acc: 0.3669 - val_loss: 1.5062 - val_acc: 0.

KeyboardInterrupt: ignored

# Prediction 

In [0]:
y_pred = classifier.predict(X_test18)
y_pred = np.argmax(y_pred, axis=1)

y_real = np.argmax(y_test18, axis=1)
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_real, y_pred)
cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

acc_test = classifier.evaluate(X_test18, y_test18)[1]
#acc_train = classifier.evaluate(X_train, y_train)[1]

print("Acc Test : ", acc_test)
#print("Acc Train : ", acc_train)


# Plot Confusion Matrix

In [0]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

modulation_order = []
modulation_order_dict = dict()

for key,value in output.items():
    modulation_order_dict[np.argmax(value)] = str(key)
    
for i in range(8):
    modulation_order.append(modulation_order_dict[i])
    
    
    
cmDataFrame = pd.DataFrame(cm_norm, index=modulation_order, columns = modulation_order)
plt.figure(figsize=(6, 5))
ax = sns.heatmap(cmDataFrame, annot=True, annot_kws={"size": 8}, fmt='.2f', linewidths=.5, cmap="Greens")

plt.title(f"CNN Confusion Matrix (SNR={snr})")
plt.xlabel("Predicted label  \n\n TrainAcc={:.2}, TestAcc={:.2}".format(acc_train,acc_test), fontsize=8)
plt.ylabel("True lable", fontsize=8)
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",rotation_mode="anchor", fontsize=8)
plt.setp(ax.get_yticklabels(), fontsize=8)
fig = ax.get_figure()